In [66]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas
import xarray
import cftime
import tensorflow as tf

In [67]:
datapath = 'nino34_monthly.nc'
nino34 = xarray.open_dataset(datapath, decode_times = False)
print(nino34)
nino34 = np.array(nino34['nino34'].values)

<xarray.Dataset>
Dimensions:         (bounds: 2, month: 12, time: 7800)
Coordinates:
  * time            (time) float64 15.5 45.0 74.5 ... 2.372e+05 2.372e+05
Dimensions without coordinates: bounds, month
Data variables:
    nino34          (time) float64 ...
    time_bnds       (time, bounds) float64 ...
    areacello       float32 ...
    days_per_month  (month) int32 ...


In [68]:
def ONI(nino34, m = 3):
    oni = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        oni[i] = np.mean(nino34[max(0, (i - m + 1)) : min((i + 1), length)])
    return oni

In [69]:
oni = ONI(nino34)

In [70]:
def climatology(nino34):
    clm = np.zeros(12)
    length = nino34.shape[0]
    for month in range(12):
        section = [12 * i + month for i in range(length // 12)]
        clm[month] = np.mean(nino34[section])
    return clm

In [71]:
clm = climatology(nino34)

In [72]:
def SST_anomaly(nino34, clm):
    anm = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        anm[i] = nino34[i] - clm[i % 12]
    return anm

In [73]:
anm = SST_anomaly(nino34, clm)
oanm = ONI(anm)

# Data Preparation

In [74]:
T = 7      # prediction timeline
H = 32   # history used for prediction
include_month = 1           # 1 if we use the month as a feature, 0 otherwise
threshold = 0.5         
signal = np.array(nino34[12 * 50:])   # data used for training/testing
length = signal.shape[0]    # number of data points

mean = np.mean(signal)
std = np.std(signal)

signal = (signal - mean) / std

In [75]:
size = 7000
sample_len = 350
n_samples = size // sample_len

In [76]:
np.random.seed(0)
        
# create the 'history matrix'
data = np.ndarray((n_samples, sample_len, H + include_month))
for i in range(n_samples):
    for j in range(sample_len):
        t = sample_len * i + j
        if(include_month == False):
             data[i,j] = signal[t:(t + H)]
        else:
            data[i,j] = np.append(signal[t:(t + H)], (t + H + T) % 12 / 12)

labels = np.ndarray((n_samples, sample_len), dtype = int)
        
for i in range(n_samples):
    for j in range(sample_len):
        t = sample_len * i + j
        labels[i, j] = oanm[t + H + T]

split = n_samples // 10      
shuffle = np.random.permutation(n_samples)
train_ind = np.array(shuffle[0: 8 * split])
val_ind = np.array(shuffle[8 * split: 9 * split])
test_ind = np.array(shuffle[9 * split: size])

train = np.array(data[train_ind])
train_labels = np.array(labels[train_ind])

val = np.array(data[val_ind])
val_labels = np.array(labels[val_ind])

test = np.array(data[test_ind])
test_labels = np.array(labels[test_ind])

In [101]:
(d, N1, N2, N3) = (0.3, 768, 768, 384)
epochs = 100
        
acc = 0
loss = 0


for n_neurons in range(10, 100, 10):
    for l in ['mse', 'mae']:

        model = tf.keras.models.Sequential([
                  tf.keras.layers.LSTM(n_neurons),
                  tf.keras.layers.Dense(1)
                ])

        model.reset_states()

        model.compile(optimizer='adam',
                loss = l,
                metrics=['mse', 'mae'])

        model.fit(train, train_labels, batch_size=n_samples, epochs = epochs, verbose = 0)
        (loss, mse, mae) = model.evaluate(val, val_labels, verbose = 0)

        print(n_neurons, l, ":", mse, mae)

10 mse : 0.4439932703971863 0.4565284252166748
10 mae : 0.28593870997428894 0.3852995038032532
20 mse : 0.2769436836242676 0.3629114627838135
20 mae : 0.2807210683822632 0.3219720125198364
30 mse : 0.34416961669921875 0.38427823781967163
30 mae : 0.32868683338165283 0.31550246477127075
40 mse : 0.3221272826194763 0.36283326148986816
40 mae : 0.27166908979415894 0.3132445216178894
50 mse : 0.32102036476135254 0.30800414085388184
50 mae : 0.357069194316864 0.3703349828720093
60 mse : 0.3003605604171753 0.34306246042251587
60 mae : 0.3151930570602417 0.2767164707183838
70 mse : 0.271484375 0.32294368743896484
70 mae : 0.28286606073379517 0.31111547350883484
80 mse : 0.30231577157974243 0.31452441215515137
80 mae : 0.3346917927265167 0.3304855227470398
90 mse : 0.34859979152679443 0.3579505681991577
90 mae : 0.3212650418281555 0.287781298160553
